In [1]:
# Parameters
image_path = "C:\\Users\\Administrator\\Desktop\\PID_PROJECT\\data\\test\\images\\test_pid.png"
outputs_dir = "C:\\Users\\Administrator\\Desktop\\PID_PROJECT\\outputs"
data_dir = "C:\\Users\\Administrator\\Desktop\\PID_PROJECT\\data\\dummy"


In [2]:
from google import genai
import os
from dotenv import load_dotenv
import json
from pathlib import Path
load_dotenv()
client = genai.Client(api_key=os.getenv('GOOGLEAI_API_KEY'))

In [3]:
outputs_dir = Path(outputs_dir)
data_dir = Path(data_dir)

readings_path   = data_dir / "values.json"
thresholds_path = data_dir / "threshold.json"
anomalies_path  = outputs_dir / "anomalies.json"
path_edges_path = outputs_dir / "path_edges.json"
components_path = outputs_dir / "components.json"

with open(readings_path, encoding="utf-8") as f:
    readings = json.load(f)
with open(thresholds_path, encoding="utf-8") as f:
    thresholds = json.load(f)
with open(anomalies_path,  encoding="utf-8") as f:
    anomalies = json.load(f)
with open(path_edges_path, encoding="utf-8") as f:
    path_edges = json.load(f)
with open(components_path, encoding="utf-8") as f:
    components = json.load(f)

In [4]:
prompt = f"""
You are a PID system diagnostics assistant.

Components of the PID system (they are connected with pipes):
{json.dumps(components)}
Network/connection of components of the PID system:
{json.dumps(path_edges)}
Anomalies detected:
{json.dumps(anomalies)}

Task:
1. Summarize the system: list components and connections.
2. Describe the anomaly detected.
3. Explain potential root causes using nearby components. (max 2 reasons)
4. Recommend inspection or remediation steps. (max 2 recommendations, keep it simple and clear)

Return only plain text.
"""

In [5]:
# Generate diagnosis
response = client.models.generate_content(
    model='gemini-2.5-flash',
    contents=prompt
)
(outputs_dir / "diagnosis.md").write_text(f"# Diagnosis\n\n{response.text}\n", encoding="utf-8")
print(response.text)


System Summary:
Components:
*   pressure indicator: S15002, S15006
*   valve: V15002, V15011, V15028, V15029
Connections:
*   V15002 is connected to S15002
*   S15002 is connected to V15011
*   V15011 is connected to V15028
*   V15028 is connected to S15006
*   S15006 is connected to V15029

Anomaly Detected:
The pressure indicator S15006 reported an observed value of 498.7 psi at 2025-10-15T10:13:00+00:00. This reading is significantly higher than its expected operational range of 101.0 - 104.0 psi.

Potential Root Causes:
1.  **Sensor Malfunction:** The pressure indicator S15006 itself may be faulty or out of calibration, leading to an inaccurately high pressure reading.
2.  **Valve Malfunction:** The upstream valve V15028 could be malfunctioning (e.g., stuck open or partially open), allowing excessive pressure to flow to S15006, or the downstream valve V15029 might be partially or fully closed, creating a pressure buildup at S15006.

Recommended Inspection/Remediation Steps:
1.  Ins